In [1]:
!pip install pysam
!pip install tqdm
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.4 MB/s eta 0:00:00


In [2]:
from tqdm import tqdm

In [7]:
import pysam

import random

def extract_sequences_from_fasta(fasta_file, tsv_file, output_file, num_lines=1000):
    # Leggi le prime num_lines righe random dal file TSV
    with open(tsv_file, 'r') as tsv:
        # Salta l'intestazione
        next(tsv)
        lines = random.sample(tsv.readlines(), num_lines)

    with open(output_file, 'w') as output:
        # Apri il file FASTA
        with pysam.FastaFile(fasta_file) as fasta:
            for line in tqdm(lines, desc='Elaborazione'):
                parts = line.strip().split('\t')
                chrom = parts[0]
                start = int(parts[1])
                end = int(parts[2])

                # Estrai la sequenza in base alle posizioni
                sequence = fasta.fetch(chrom, start, end)

                # Scrivi la sequenza in un file di output
                output.write(f'>{chrom}:{start}-{end}\n')
                output.write(sequence + '\n')

# Specifica i percorsi dei file
fasta_file = '/content/drive/MyDrive/hg19.fa'
tsv_file = '/content/drive/MyDrive/esoni_noduplicati.tsv'
output_file = '/content/drive/MyDrive/esoni_targettato_genoma_triplette_new.fa'


In [8]:
from Bio import SeqIO
from collections import Counter

# Definisci una funzione per generare tutte le triplette possibili
def generate_triplets():
    bases = "ACGT"
    triplets = [base1 + base2 + base3 for base1 in bases for base2 in bases for base3 in bases]
    return triplets

In [9]:
triplets_list=[]

In [10]:
# Esegui l'estrazione delle sequenze
for i in range(1000):
  extract_sequences_from_fasta(fasta_file, tsv_file, output_file,num_lines=1000)
  # Inizializza un contatore per le triplette
  triplet_counter = Counter(generate_triplets())
  # Esegui il conteggio delle triplette nel file FASTA
  for record in SeqIO.parse(output_file, "fasta"):
    sequence = str(record.seq).upper()  # Converte la sequenza in maiuscolo
    for i in range(0, len(sequence) - 2, 3):
      triplet = sequence[i:i + 3]
      if triplet in triplet_counter:
        triplet_counter[triplet] += 1
  for key in triplet_counter:
    triplet_counter[key] -= 1
  triplets_list.append(triplet_counter)

Elaborazione: 100%|██████████| 1000/1000 [00:00<00:00, 6523.17it/s]


In [11]:
# Ottenere i valori associati a target_key in ciascun Counter
values_for_AAA = [counter['AAA'] for counter in triplets_list]

In [12]:
# Ottenere tutte le chiavi univoche dai contatori
all_keys = set().union(*triplets_list)

# Creare un dizionario per i valori associati a ciascuna chiave
values_for_all_keys = {key: [counter[key] for counter in triplets_list] for key in all_keys}


In [13]:
len(all_keys)

64

In [14]:
import numpy as np


In [15]:
# Calcolare mediana e varianza per ciascuna chiave
result = {}
for key, values in values_for_all_keys.items():
    median = np.median(values)
    variance = np.var(values)
    result[key] = {"median": median, "variance": variance}


In [16]:
result

{'ATC': {'median': 1432.5, 'variance': 16252.108096},
 'ATA': {'median': 1496.0, 'variance': 34913.0664},
 'ATT': {'median': 2048.0, 'variance': 57374.476384},
 'CTA': {'median': 1153.0, 'variance': 14765.228079},
 'TAA': {'median': 1642.0, 'variance': 41142.268575999995},
 'TCT': {'median': 2505.5, 'variance': 47906.530464},
 'TTG': {'median': 1955.5, 'variance': 35603.128311},
 'GGA': {'median': 2388.0, 'variance': 34265.304775},
 'CAT': {'median': 2012.0, 'variance': 34491.406559},
 'GGT': {'median': 1650.5, 'variance': 18084.761439},
 'TCA': {'median': 2232.0, 'variance': 38803.145344},
 'AAA': {'median': 3225.0, 'variance': 139391.485024},
 'CTT': {'median': 2317.0, 'variance': 43048.141504},
 'AGC': {'median': 2103.0, 'variance': 26598.182544},
 'TGC': {'median': 2026.0, 'variance': 26268.543590999998},
 'TGT': {'median': 2167.0, 'variance': 45444.5579},
 'CTC': {'median': 2234.0, 'variance': 33585.732599},
 'CCA': {'median': 2666.5, 'variance': 43259.721399},
 'AAT': {'median': 